# [LEGALST-123] Lab 10: Folium Plugins



This lab will provide an introduction to extra plugins in the folium package: Search and Draw.

*Estimated Time: 30 minutes*

---

### Table of Contents

[The Data](#section data)<br>

[Context](#section context)<br>

1 - [Search](#section 1)<br>

2 - [Draw](#section 2)<br>





**Dependencies:**

In [12]:
import json
import os
import pandas as pd
import numpy as np

In [2]:
!pip install folium --upgrade
import folium
from folium import plugins
from folium.plugins import Draw
from folium.plugins import HeatMap

Requirement not upgraded as not directly required: six in c:\programdata\anaconda3\lib\site-packages (from folium) (1.11.0)
Requirement not upgraded as not directly required: jinja2 in c:\programdata\anaconda3\lib\site-packages (from folium) (2.10)
Requirement not upgraded as not directly required: branca>=0.3.0 in c:\programdata\anaconda3\lib\site-packages (from folium) (0.3.1)
Requirement not upgraded as not directly required: requests in c:\programdata\anaconda3\lib\site-packages (from folium) (2.18.4)
Requirement not upgraded as not directly required: numpy in c:\programdata\anaconda3\lib\site-packages (from folium) (1.14.3)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in c:\programdata\anaconda3\lib\site-packages (from jinja2->folium) (1.0)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in c:\programdata\anaconda3\lib\site-packages (from requests->folium) (3.0.4)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in c

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
# %load search.py

from __future__ import (absolute_import, division, print_function)

from branca.element import CssLink, Figure, JavascriptLink, MacroElement

from jinja2 import Template


class Search(MacroElement):
    """
    Adds a search tool to your map.

    Parameters
    ----------
    data: str/JSON
        GeoJSON strings
    search_zoom: int
        zoom level when searching features, default 12
    search_label: str
        label to index the search, default 'name'
    geom_type: str
        geometry type, default 'Point'
    position: str
        Change the position of the search bar, can be:
        'topleft', 'topright', 'bottomright' or 'bottomleft',
        default 'topleft'

    See https://github.com/stefanocudini/leaflet-search for more information.

    """
    def __init__(self, data, search_zoom=12, search_label='name', geom_type='Point', position='topleft'):
        super(Search, self).__init__()
        self.position = position
        self.data = data
        self.search_label = search_label
        self.search_zoom = search_zoom
        self.geom_type = geom_type

        self._template = Template("""
        {% macro script(this, kwargs) %}

            var {{this.get_name()}} = new L.GeoJSON({{this.data}});

            {{this._parent.get_name()}}.addLayer({{this.get_name()}});

            var searchControl = new L.Control.Search({
                layer: {{this.get_name()}},
                propertyName: '{{this.search_label}}',
            {% if this.geom_type == 'Point' %}
                initial: false,
                zoom: {{this.search_zoom}},
                position:'{{this.position}}',
                hideMarkerOnCollapse: true
            {% endif %}
            {% if this.geom_type == 'Polygon' %}
                marker: false,
                moveToLocation: function(latlng, title, map) {
                var zoom = {{this._parent.get_name()}}.getBoundsZoom(latlng.layer.getBounds());
                    {{this._parent.get_name()}}.setView(latlng, zoom); // access the zoom
                }
            {% endif %}
                });
                searchControl.on('search:locationfound', function(e) {

                    e.layer.setStyle({fillColor: '#3f0', color: '#0f0'});
                    if(e.layer._popup)
                        e.layer.openPopup();

                }).on('search:collapsed', function(e) {

                    {{this.get_name()}}.eachLayer(function(layer) {   //restore feature color
                        {{this.get_name()}}.resetStyle(layer);
                    });
                });
            {{this._parent.get_name()}}.addControl( searchControl );

        {% endmacro %}
        """)  # noqa

    def render(self, **kwargs):
        super(Search, self).render()

        figure = self.get_root()
        assert isinstance(figure, Figure), ('You cannot render this Element '
                                            'if it is not in a Figure.')

        figure.header.add_child(
            JavascriptLink('https://cdn.jsdelivr.net/npm/leaflet-search@2.3.6/dist/leaflet-search.min.js'),  # noqa
            name='Leaflet.Search.js'
        )

        figure.header.add_child(
            CssLink('https://cdn.jsdelivr.net/npm/leaflet-search@2.3.6/dist/leaflet-search.min.css'),  # noqa
            name='Leaflet.Search.css'
        )


---

## The Data <a id='data'></a>
---

In [4]:
# Searchable US states
m = folium.Map(location = [39.8283, -98.5795], zoom_start=5)
m
states = os.path.join('data', 'search_states.json')
states_data = json.load(open(states))
states_data
# with open(os.path.join('data', 'search_states.json')) as f:
#     states = json.loads(f.read())


folium.GeoJson(
    states_data,
    style_function=lambda feature: {
        'fillColor': '#800026' if feature['properties']['density'] > 1000
            else '#BD0026' if feature['properties']['density'] > 500
            else '#E31A1C' if feature['properties']['density'] > 200
            else '#FC4E2A' if feature['properties']['density'] > 100
            else '#FD8D3C' if feature['properties']['density'] > 50
            else '#FEB24C' if feature['properties']['density'] > 20
            else '#FED976' if feature['properties']['density'] > 10
            else 'FFEDA0',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)



# Searchable Roman location markers
rome_bars = os.path.join('data', 'search_bars_rome.json')
rome = json.load(open(rome_bars))
    
# Berkeley PD call report location coordinates
with open('data/call_locs.txt') as f:
    coords = np.fromfile(f).tolist()
    call_locs = [[coords[i], coords[i + 1]] for i in range(0, len(coords), 2)]

m

In [5]:
states_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '01',
   'properties': {'name': 'Alabama',
    'density': 94.65,
    'color': '#FD8D3C',
    'style': {'fillColor': '#FD8D3C',
     'color': 'black',
     'weight': 2,
     'dashArray': '5, 5'},
    'highlight': {}},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.359296, 35.00118],
      [-85.606675, 34.984749],
      [-85.431413, 34.124869],
      [-85.184951, 32.859696],
      [-85.069935, 32.580372],
      [-84.960397, 32.421541],
      [-85.004212, 32.322956],
      [-84.889196, 32.262709],
      [-85.058981, 32.13674],
      [-85.053504, 32.01077],
      [-85.141136, 31.840985],
      [-85.042551, 31.539753],
      [-85.113751, 31.27686],
      [-85.004212, 31.003013],
      [-85.497137, 30.997536],
      [-87.600282, 30.997536],
      [-87.633143, 30.86609],
      [-87.408589, 30.674397],
      [-87.446927, 30.510088],
      [-87.37025, 30.427934],
      [-87.518128, 30.280057],
      [-87.6550

---

## Plugins <a id='data'></a>
Now that we're familiar with several different types of Folium mapping, we make our maps more functional with two additional **plug-ins**: Folium package components that add new features. Today, we'll work with **Search** and **Draw**.


---


## Search  <a id='section 1'></a>

Most map applications you've used likely contained a search function. Here's how to add one using Folium.

Create a map of the United States. Next, create a Search object by calling its constructor function, then add the Search to your Map using `add_too`- just like you've done with many other Folium modules. 

`Search(...)` needs to be given some searchable GeoJSON-formatted data. Here, you can use `states`. 

Hint: Remember, the `add_too` function call looks something like `<thing you want to add>.add_to(<place you want to add it to>)`

In [6]:
# code from https://github.com/python-visualization/folium/blob/master/examples/plugin-Search.ipynb
m = folium.Map(
    location=[37.8, -96],
    zoom_start=3,
)

Search(states_data, search_zoom=10, geom_type='Polygon').add_to(m)


m

You can also set optional parameters such as the location of the bar. We loaded the code for Search in an earlier cell: look at the 'parameters' section to see what options you can change, then see what happens when you change `geom_type`, `search_zoom`, and `position` on your states search map. 

In addition to searching for whole geographic areas, we can also search for specific locations. Create another searchable map in the cell below, this time using [41.9, 12.5] for the map coordinates and `rome` for your search data. You'll want to set a high `search_zoom`. How many cafes can you find?

In [7]:
# code from https://github.com/python-visualization/folium/blob/master/examples/plugin-Search.ipynb

m = folium.Map(
    location=[41.9, 12.5],
    tiles='cartodbpositron',
    zoom_start=12,
)


Search(rome, search_zoom=50).add_to(m)

m



Folium's Search is currently pretty limited, but it can be useful when you have the right set of GeoJSON data. 
1. Go to http://geojson.io, a tool for creating and downloading GeoJSON (the interface might look familiar: they use Leaflet, the same software Folium is built around).  
2. Create ten markers any where you'd like. Click on each marker and name them by adding a row with 'name' in the left column and the marker's name in the right column.
3. Save your markers in a GeoJSON file by using the save menu in the upper right.
4. Download your GeoJSON file to your local directory
5. Open your file in this notebook and create a searchable map using your markers. Hint: there's an example of code for opening a GeoJSON file earlier in this notebook.


In [8]:
with open(os.path.join('data', 'map.geojson')) as f:
    my_map = json.loads(f.read())
    
world = folium.Map()
Search(my_map).add_to(world)
world

FileNotFoundError: [Errno 2] No such file or directory: 'data\\map.geojson'

---

## Draw <a id='section 2'></a>

A **Draw** tool can be a great way to interactively add, change, or highlight features on your map. Adding it is  simple:

1. Create a Map 
2. Create a Draw object by calling the `Draw()` constructor function 
3. Add your Draw object to your map using `add_to`



Folium's Draw has a lot of features. Let's try a few.

- Draw a polygon representing the [Bermuda Triangle](https://en.wikipedia.org/wiki/Bermuda_Triangle)
- The Deepwater Horizon oil spill in 2010 occurred in the nearby Gulf of Mexico. Look up the exact coordinates, add a marker for the center, and draw two circles: one representing the size of the spill at its smallest, and one at its largest.

In [9]:
folium.Map([-70.3125, 27.059125784374068])


- The Proclaimers say [they would walk 500 miles and they would walk 500 more just to be the man who walked 1000 miles to fall down at your door](https://en.wikipedia.org/wiki/I%27m_Gonna_Be_(500_Miles). Create a map with a marker at the Proclaimers' birthplace in Edinburgh, Scotland. Then make a circle with a radius of 1000 miles (about 1600 kilometers) centered at Edinburgh to see if the Proclaimers can reach your door.
- Vanessa Carlton lives in Nashville, Tennessee and says that [you know she'd walk 1000 miles if she could just see you](https://en.wikipedia.org/wiki/A_Thousand_Miles). Use the edit layers button to move your circle to be centered at Nashville and see if you're in Vanessa's range. Note: the circle changes size slightly when you move it south. Why?



In [10]:
map3 = folium.Map([55.953251, -3.188267], zoom_start=2)
folium.Marker([55.953251, -3.188267], popup = "The Proclaimers").add_to(map3)

folium.Marker([36.174465, -86.767960], popup = "Vanessa Carlton").add_to(map3)

map3

- Recreate the HeatMap you made on Tuesday using the Berkeley Police Department call location data (the call coordinates are stored in the variable `call_locs`). Add the Draw tool, then draw a polyline representing the route from Barrows Hall to your home that passes through the fewest call locations. How long is it? How does it compare to the shortest possible route?


In [11]:
berk_map = folium.Map(location=(37.87, -122.27), zoom_start=13) # Solution
# Create a Heatmap with the call data.
heatmap = HeatMap(call_locs, radius = 10) # Solution

# Add it to your Berkeley map.
berk_map.add_child(heatmap)

Draw().add_to(berk_map)
berk_map

Visualization is a powerful way to understand and communicate data. For each Draw tool, think of one data analysis project that would be a good use of the tool:

**ANSWER:**

Congratulations; you're a folium expert!

---
Notebook developed by: Keeley Takimoto; 2019 revision by Chakshu Hurria

Data Science Modules: http://data.berkeley.edu/education/modules
